In [ ]:
import openai
import matplotlib.pyplot as plt
import nltk
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
nltk.download('punkt')

In [ ]:
openai.api_key = "####"

# Pre-Defined Testing Prompts for React Code Generation:


In [ ]:
prompts = {
    "Prompt 1": "Write a minimal React functional component based on the following description:",
    "Prompt 2": "Generate a JSX code snippet for the given UI description:",
    "Prompt 3": "Create a simple React component that matches the description below:",
    "Prompt 4": "Provide clean and minimal React (JSX) code for:",
    "Prompt 5": "Write a concise React component using hooks if needed for the following UI:",
    "Prompt 6": "Develop a simple React UI element as per the following description:",
    "Prompt 7": "Create a TypeScript React component for the given description:",
    "Prompt 8": "Write a JSX code snippet optimized for readability based on this UI description:",
    "Prompt 9": "Given the following description, build a functional React component:",
    "Prompt 10": "Generate a basic React structure in JSX to match the following UI description:",
}

#Sample Descriptions and Target Codes (Dashboard Components):

In [ ]:
descriptions = [
    "A line chart displaying sales data with tooltips.",
    "A dashboard card showing a user's total earnings.",
    "A bar chart comparing monthly active users.",
    "A pie chart showing market share distribution.",
    "A KPI widget displaying the number of new customers this month.",
]

In [ ]:
target_codes = [
    "<LineChart data={data}><Line type='monotone' dataKey='sales' /><Tooltip /></LineChart>",
    "<div className='card'><h2>Total Earnings</h2><p>$25,000</p></div>",
    "<BarChart data={data}><Bar dataKey='users' fill='#8884d8' /></BarChart>",
    "<PieChart><Pie data={data} dataKey='value' nameKey='name' cx='50%' cy='50%' outerRadius={80} fill='#82ca9d' label /></PieChart>",
    "<div className='kpi-widget'><h3>New Customers</h3><p>120</p></div>",
]

#Sending Prompt to LLM:

In [ ]:
def query_openai(prompt_text, input_text):
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"{prompt_text}\n{input_text}"},
        ],
        temperature=0.2,
        max_tokens=300
    )
    return response['choices'][0]['message']['content'].strip()

#BLEU Score Calculation:

In [ ]:
results = {}
bleu_scores = {prompt_name: [] for prompt_name in prompts.keys()}

total_steps = len(prompts) * len(descriptions)
current_step = 0

for prompt_name, prompt_template in prompts.items():
    generated_codes = []
    for description in descriptions:
        code = query_openai(prompt_template, description)
        generated_codes.append(code)
        current_step += 1
    results[prompt_name] = generated_codes

In [ ]:
for prompt_name, codes in results.items():
    for generated, target_code in zip(codes, target_codes):
        reference = target_code.split()
        candidate = generated.split()
        bleu = sentence_bleu([reference], candidate)
        bleu_scores[prompt_name].append(bleu)

average_bleu = {k: sum(v)/len(v) for k, v in bleu_scores.items()}

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(average_bleu.keys(), average_bleu.values())
plt.ylabel("Average BLEU Score")
plt.title("Prompt Template vs. BLEU Score for React Code Generation")
plt.xticks(rotation=45, ha='right')
plt.ylim(0, 1)
plt.tight_layout()
plt.show()

In [ ]:
for prompt_name, scores in bleu_scores.items():
    print(f"\n{prompt_name}")
    for idx, score in enumerate(scores):
        print(f"Component {idx+1}: BLEU Score = {score:.2f}")

In [ ]:
best_prompt = max(average_bleu, key=average_bleu.get)
print(f"\n🏆 Best Performing Prompt: {best_prompt} with Avg BLEU = {average_bleu[best_prompt]:.2f}")